In [4]:
from transformers import ViTForImageClassification, TrainingArguments, Trainer
from transformers import ViTFeatureExtractor
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
from PIL import Image
import json
import os

c:\Users\Shreyas Holkar\deep cnn image classifier\edi\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
pip uninstall torch

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
print("Is CUDA available?", torch.cuda.is_available())
print("CUDA device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


Is CUDA available? False
CUDA device: None


In [5]:
base_directory = "E:/Dataset/timeday_dataset"

In [6]:
directory_images_nightd = os.path.join(base_directory, "agv_night_dark")
directory_annotations_nightd = os.path.join(base_directory, "agv_night_dark_labels")

image_files = os.listdir(directory_images_nightd)
# Print the total count of image files
print(f"Total number of files in base_directory_images: {len(image_files)}")
# Optionally, print the files themselves (can be omitted if the list is too long)
#print("Files in base_directory_images:", image_files)

txt_files = os.listdir(directory_annotations_nightd)
print(f"Total number of files in base_directory_images: {len(txt_files)}")
#print("Files in base_directory_images:", txt_files)

Total number of files in base_directory_images: 340
Total number of files in base_directory_images: 340


In [7]:
directory_images_nightl = os.path.join(base_directory, "agv_night_light")
directory_annotations_nightl = os.path.join(base_directory, "agv_night_light_labels")

image_files = os.listdir(directory_images_nightl)
# Print the total count of image files
print(f"Total number of files in base_directory_images: {len(image_files)}")
# Optionally, print the files themselves (can be omitted if the list is too long)
#print("Files in base_directory_images:", image_files)

txt_files = os.listdir(directory_annotations_nightl)
print(f"Total number of files in base_directory_images: {len(txt_files)}")
#print("Files in base_directory_images:", txt_files)

Total number of files in base_directory_images: 899
Total number of files in base_directory_images: 899


In [8]:
directory_images_misc = os.path.join(base_directory, "misc")
directory_annotations_misc_labels = os.path.join(base_directory, "misc_labels")

image_files = os.listdir(directory_images_misc)
# Print the total count of image files
print(f"Total number of files in base_directory_images: {len(image_files)}")
# Optionally, print the files themselves (can be omitted if the list is too long)
#print("Files in base_directory_images:", image_files)

txt_files = os.listdir(directory_annotations_misc_labels)
print(f"Total number of files in base_directory_images: {len(txt_files)}")
#print("Files in base_directory_images:", txt_files)

Total number of files in base_directory_images: 4030
Total number of files in base_directory_images: 4030


In [9]:
'''Debugging: Check if filenames are matching
image_basenames = [os.path.splitext(f)[0] for f in image_files]  # Remove extension
json_basenames = [os.path.splitext(f)[0] for f in txt_files]    # Remove extension

# Find missing JSON files
missing_json_files = [f for f in image_basenames if f not in json_basenames]

print(f"Total image basenames: {len(image_basenames)}")
print(f"Total text basenames: {len(json_basenames)}")
print("Missing text annotations:", missing_json_files)'''

'Debugging: Check if filenames are matching\nimage_basenames = [os.path.splitext(f)[0] for f in image_files]  # Remove extension\njson_basenames = [os.path.splitext(f)[0] for f in txt_files]    # Remove extension\n\n# Find missing JSON files\nmissing_json_files = [f for f in image_basenames if f not in json_basenames]\n\nprint(f"Total image basenames: {len(image_basenames)}")\nprint(f"Total text basenames: {len(json_basenames)}")\nprint("Missing text annotations:", missing_json_files)'

In [10]:
from torch.utils.data import Dataset
from PIL import Image
import os

class CustomDataset(Dataset):
    def __init__(self, image_dir, annotation_dir, transform=None, num_classes=15):
        self.image_dir = image_dir
        self.annotation_dir = annotation_dir
        self.image_filenames = os.listdir(image_dir)
        self.transform = transform
        self.num_classes = num_classes

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_filename = self.image_filenames[idx]
        image_path = os.path.join(self.image_dir, image_filename)
        annotation_path = os.path.join(self.annotation_dir, os.path.splitext(image_filename)[0] + ".txt")

        # Load the image
        image = Image.open(image_path).convert("RGB")

        # Read the annotations and convert them to a multi-label vector
        with open(annotation_path, 'r') as f:
            annotations = f.readlines()

        class_ids = [int(line.split()[0]) for line in annotations]
        labels = self.create_multilabel_target(class_ids, self.num_classes)

        if self.transform:
            image = self.transform(image)

        return image, labels

    @staticmethod
    def create_multilabel_target(class_ids, num_classes):
        """
        Converts a list of class IDs into a multi-label binary vector.
        """
        label_vector = [0] * num_classes
        for class_id in class_ids:
            label_vector[class_id] = 1
        return torch.tensor(label_vector, dtype=torch.float32)


In [11]:
from torchvision import transforms
from torch.utils.data import DataLoader

# Directories
image_dir = r"E:\Dataset\timeday_dataset\agv_day"
annotation_dir = r"E:\Dataset\timeday_dataset\agv_day_labels"

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG16 expects 224x224 images
    transforms.ToTensor(),
])

# Dataset and DataLoader
dataset = CustomDataset(image_dir, annotation_dir, transform=transform, num_classes=15)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [12]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

In [13]:
'''class ImageClassificationDataset(Dataset):
    def __init__(self, image_dir, annotation_dir, transform=None):
        self.image_dir = image_dir
        self.annotation_dir = annotation_dir
        self.transform = transform
        self.data = []
        
        for annotation_file in os.listdir(annotation_dir):
            if annotation_file.endswith(".txt"):
                image_name = annotation_file.replace(".txt", ".jpg")
                image_path = os.path.join(image_dir, image_name)
                file_path = os.path.join(annotation_dir, annotation_file)
                
                if not os.path.exists(image_path):
                    print(f"Skipping missing file: {image_name}")
                    continue
                
                with open(file_path, 'r') as f:
                    lines = f.readlines()
                    if not lines:
                        continue
                    
                    class_ids = [int(line.split()[0]) for line in lines]
                    label = max(set(class_ids), key=class_ids.count)
                    self.data.append((image_name, label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_name, label = self.data[idx]
        image_path = os.path.join(self.image_dir, image_name)
        image = Image.open(image_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


# Paths
image_dir = r"E:\Dataset\timeday_dataset\agv_day"
annotation_dir = r"E:\Dataset\timeday_dataset\agv_day_labels"

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset and DataLoader
dataset = ImageClassificationDataset(image_dir, annotation_dir, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
'''


<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Shreyas Holkar\AppData\Local\Temp\ipykernel_4884\1773892157.py:1: SyntaxWarning: invalid escape sequence '\D'
  '''class ImageClassificationDataset(Dataset):


'class ImageClassificationDataset(Dataset):\n    def __init__(self, image_dir, annotation_dir, transform=None):\n        self.image_dir = image_dir\n        self.annotation_dir = annotation_dir\n        self.transform = transform\n        self.data = []\n        \n        for annotation_file in os.listdir(annotation_dir):\n            if annotation_file.endswith(".txt"):\n                image_name = annotation_file.replace(".txt", ".jpg")\n                image_path = os.path.join(image_dir, image_name)\n                file_path = os.path.join(annotation_dir, annotation_file)\n                \n                if not os.path.exists(image_path):\n                    print(f"Skipping missing file: {image_name}")\n                    continue\n                \n                with open(file_path, \'r\') as f:\n                    lines = f.readlines()\n                    if not lines:\n                        continue\n                    \n                    class_ids = [int(line.sp

In [14]:
import torch.nn as nn
from torchvision import models

# Load pre-trained VGG16
model = models.vgg16(pretrained=True)

# Adjust the output layer for 15 classes
num_classes = 15
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

# Print the model to verify
print(model)

c:\Users\Shreyas Holkar\deep cnn image classifier\edi\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Shreyas Holkar\deep cnn image classifier\edi\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [56]:
import torch

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(dataloader):.4f}")


Epoch [1/10], Loss: 0.4634
Epoch [2/10], Loss: 0.3137
Epoch [3/10], Loss: 0.2744
Epoch [4/10], Loss: 0.2327
Epoch [5/10], Loss: 0.2154
Epoch [6/10], Loss: 0.1889
Epoch [7/10], Loss: 0.1773
Epoch [8/10], Loss: 0.1572
Epoch [9/10], Loss: 0.1362
Epoch [10/10], Loss: 0.1243


In [13]:
# Save the entire model
torch.save(model, "vgg16_multilabel_model.pth")

print("Model saved as vgg16_multilabel_model.pth")


Model saved as vgg16_multilabel_model.pth


In [15]:
import torch

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [16]:
import torch
# Load the saved model
model = torch.load("vgg16_multilabel_model.pth")
model.eval()  # Set the model to evaluation mode

print("Model loaded successfully!")


C:\Users\Shreyas Holkar\AppData\Local\Temp\ipykernel_4884\3801638789.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("vgg16_multilabel_model.pth")


Model loaded successfully!


In [17]:
import torch.nn.functional as F
from PIL import Image

model.eval()
with torch.no_grad():
    '''image = Image.open("E:/Dataset/timeday_dataset/Combined-Current-and-Voltage-Transformers.png").convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension'''

    image = Image.open("E:/Dataset/timeday_dataset/distribution-substation-equipment-1.jpg").convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    outputs = model(image)
    probabilities = torch.sigmoid(outputs).squeeze(0)  # Apply sigmoid for multi-label probabilities

    # Threshold to decide which classes are predicted (e.g., > 0.5)
    class_names = [
    "COpen blade disconnect switch", "Closed blade disconnect switch", "Open tandem disconnect switch", "Closed tandem disconnect switch", "Breaker", 
    "Fuse disconnect switch", "Glass disc insulator", "Porcelain pin insulator", "Muffle", "Lightning arrester", 
    "Recloser", "Power transformer", "Current transformer", "Potential transformer", "Tripolar disconnect switch"
    ]

    # Assuming 'predicted' is the output from the model (logits or probabilities)
    predicted = torch.sigmoid(outputs) > 0.5  # Convert to binary predictions for multi-label

    # Get the indices of classes predicted as 1 (positive prediction)
    predicted_indices = torch.where(predicted[0] == 1)[0]  # For a single image

    # Map indices to class names
    predicted_classes = [class_names[idx] for idx in predicted_indices.tolist()]

    # Print the predicted class names
    print("Predicted classes:", predicted_classes)

    predicted_index = [i for i, prob in enumerate(probabilities) if prob > 0.5]
    print("Predicted classes:", predicted_index)


Predicted classes: ['Closed tandem disconnect switch', 'Breaker', 'Glass disc insulator', 'Porcelain pin insulator', 'Muffle', 'Recloser', 'Current transformer']
Predicted classes: [3, 4, 6, 7, 8, 10, 12]


In [18]:
'''import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import os
import torch
import torchvision.transforms as transforms
from IPython.display import display, HTML

# GUI within Jupyter Notebook
class ImageClassifierNotebookGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Classification GUI")

        # GUI Elements
        self.label = tk.Label(root, text="Select an image for classification")
        self.label.pack(pady=10)

        self.image_label = tk.Label(root)
        self.image_label.pack(pady=10)

        self.select_button = tk.Button(root, text="Select Image", command=self.select_image)
        self.select_button.pack(pady=5)

        self.classify_button = tk.Button(root, text="Classify Image", command=self.classify_image, state=tk.DISABLED)
        self.classify_button.pack(pady=5)

        self.image_path = None

    def select_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
        if file_path:
            self.image_path = file_path
            image = Image.open(file_path)
            image.thumbnail((300, 300))
            img = ImageTk.PhotoImage(image)
            self.image_label.config(image=img)
            self.image_label.image = img
            self.classify_button.config(state=tk.NORMAL)

    def classify_image(self):
        if self.image_path:
            try:
                # Image preprocessing (update as per your model requirements)
                transform = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ])

                image = Image.open(self.image_path).convert('RGB')
                input_tensor = transform(image).unsqueeze(0)

                # Replace 'model' with your actual PyTorch model
                model.eval()  # Ensure the model is in evaluation mode
                with torch.no_grad():
                    outputs = model(input_tensor)
                    probabilities = torch.nn.functional.softmax(outputs[0], dim=0)

                # Filter classes with probabilities >= 60%
                threshold = 0.6
                results = []
                for i, prob in enumerate(probabilities):
                    if prob.item() >= threshold:
                        class_name = class_names[i]
                        results.append(f"{class_name}: {prob.item() * 100:.2f}%")

                # Display the result
                if results:
                    result_message = "\n".join(results)
                    messagebox.showinfo("Classification Results", f"Predicted Classes (>= 60%):\n{result_message}")
                else:
                    messagebox.showinfo("Classification Results", "No classes identified with >= 60% confidence.")

            except Exception as e:
                messagebox.showerror("Error", f"An error occurred: {e}")

# Run the GUI
if __name__ == "__main__":
    root = tk.Tk()
    app = ImageClassifierNotebookGUI(root)
    root.mainloop()'''


'import tkinter as tk\nfrom tkinter import filedialog, messagebox\nfrom PIL import Image, ImageTk\nimport os\nimport torch\nimport torchvision.transforms as transforms\nfrom IPython.display import display, HTML\n\n# GUI within Jupyter Notebook\nclass ImageClassifierNotebookGUI:\n    def __init__(self, root):\n        self.root = root\n        self.root.title("Image Classification GUI")\n\n        # GUI Elements\n        self.label = tk.Label(root, text="Select an image for classification")\n        self.label.pack(pady=10)\n\n        self.image_label = tk.Label(root)\n        self.image_label.pack(pady=10)\n\n        self.select_button = tk.Button(root, text="Select Image", command=self.select_image)\n        self.select_button.pack(pady=5)\n\n        self.classify_button = tk.Button(root, text="Classify Image", command=self.classify_image, state=tk.DISABLED)\n        self.classify_button.pack(pady=5)\n\n        self.image_path = None\n\n    def select_image(self):\n        file_path =

In [20]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import os
import torch
import torchvision.transforms as transforms
from IPython.display import display, HTML

# GUI within Jupyter Notebook
class ImageClassifierNotebookGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Classification GUI")

        # GUI Elements
        self.label = tk.Label(root, text="Select an image for classification")
        self.label.pack(pady=10)

        self.image_label = tk.Label(root)
        self.image_label.pack(pady=10)

        self.select_button = tk.Button(root, text="Select Image", command=self.select_image)
        self.select_button.pack(pady=5)

        self.classify_button = tk.Button(root, text="Classify Image", command=self.classify_image, state=tk.DISABLED)
        self.classify_button.pack(pady=5)

        self.image_path = None

    def select_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
        if file_path:
            self.image_path = file_path
            image = Image.open(file_path)
            image.thumbnail((300, 300))
            img = ImageTk.PhotoImage(image)
            self.image_label.config(image=img)
            self.image_label.image = img
            self.classify_button.config(state=tk.NORMAL)

    def classify_image(self):
        if self.image_path:
            try:
                # Image preprocessing (update as per your model requirements)
                transform = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ])

                image = Image.open(self.image_path).convert('RGB')
                input_tensor = transform(image).unsqueeze(0)

                # Replace 'model' with your actual PyTorch model
                model.eval()  # Ensure the model is in evaluation mode
                with torch.no_grad():
                    outputs = model(input_tensor)
                    probabilities = torch.nn.functional.softmax(outputs[0], dim=0)

                # Get top classes and their probabilities
                top_probabilities, top_indices = torch.topk(probabilities, len(class_names))

                results = []
                for i in range(len(top_indices)):
                    class_id = top_indices[i].item()
                    class_name = class_names[class_id]
                    prob = top_probabilities[i].item()
                    results.append(f"{class_name}: {prob:.4f}")

                # Display the result
                result_message = "\n".join(results)
                messagebox.showinfo("Classification Results", f"Predicted Classes:\n{result_message}")

            except Exception as e:
                messagebox.showerror("Error", f"An error occurred: {e}")

# Run the GUI
if __name__ == "__main__":
    root = tk.Tk()
    app = ImageClassifierNotebookGUI(root)
    root.mainloop()